In [95]:
import os
import pandas as pd
from typing import List, Union

DATA_DIR = '../data'
RAW_DATASET = 'raw/iris.txt'
CLEANED_DATASET = 'processed/cleaned.csv'
NON_COMPLIANT_DATASET = 'processed/non_compliant.csv'

In [96]:
def categorise_data(row) -> str:
    if row['sepal_length'] < 4.3 or row['sepal_length'] > 7.9:
        return 'sepal_length must be in range (4.3, 7.9)'
    elif row['sepal_width'] < 1.0 or row['sepal_width'] > 4.4:
        return 'sepal_width must be in range (1.0, 4.4)'
    elif row['petal_length'] < 1.0 or row['petal_length'] > 6.9:
        return 'petal_length must be in range (1.0, 6.9)'
    elif row['petal_width'] < 0.1 or row['petal_width'] > 2.5:
        return 'petal_length must be in range (0.1, 2.5)'
    elif row['classEncoder'] not in [0, 1, 2]:
        return 'classEncoder must assume one of the values: 0, 1, 2'
    elif row['class'] not in ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']:
        return "class must assume one of the values: 'Iris-setosa', 'Iris-versicolor' ou 'Iris-virginica'"
    else:
        return None
    
def load_data(file: str) -> pd.DataFrame:
    return pd.read_csv(file, header=0, index_col=False)

def write_data(file: str, df: pd.DataFrame, sep: str=',') -> None:
    df.to_csv(file, sep=sep)

def clean_data(df: pd.DataFrame, returnBadResults=True) -> Union[pd.DataFrame, pd.DataFrame]:
    cleaned_data_df = df[
        (df['sepal_length'].between(4.3, 7.9)) &
        (df['sepal_width'].between(1.0, 4.4)) &
        (df['petal_length'].between(1.0, 6.9)) &
        (df['petal_width'].between(0.1, 2.5)) &
        (df['classEncoder'].isin([0, 1, 2])) &
        (df['class'].isin(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']))
    ]
    
    non_conformant_df = pd.DataFrame() 
    
    if returnBadResults:
        non_conformant_df = df[~df.apply(tuple, 1).isin(cleaned_data_df.apply(tuple, 1))]
        non_conformant_df.loc[:, 'messageError'] = non_conformant_df.apply(lambda row: categorise_data(row), axis=1)

        return (cleaned_data_df, non_conformant_df)
    
    return (cleaned_data_df, non_conformant_df)

In [97]:
file = os.path.join(DATA_DIR, RAW_DATASET)
df_iris = load_data(file)
cleaned_df, non_conformant_df = clean_data(df=df_iris, returnBadResults=True)

write_data(file=os.path.join(DATA_DIR, CLEANED_DATASET), df=cleaned_df)

write_data(file=os.path.join(DATA_DIR, NON_COMPLIANT_DATASET), df=non_conformant_df)
non_conformant_df.head()

/tmp/ipykernel_7058/3198564530.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_conformant_df.loc[:, 'messageError'] = non_conformant_df.apply(lambda row: categorise_data(row), axis=1)


,sepal_length,sepal_width,petal_length,petal_width,class,classEncoder,messageError
5,15.0,3.9,1.7,0.4,Iris-setosa,0,"sepal_length must be in range (4.3, 7.9)"
15,5.7,15.0,1.5,0.4,Iris-setosa,0,"sepal_width must be in range (1.0, 4.4)"
22,4.6,15.0,1.0,0.2,Iris-setosa,0,"sepal_width must be in range (1.0, 4.4)"
25,5.0,3.0,1.6,20.0,Iris-setosa,0,"petal_length must be in range (0.1, 2.5)"
28,5.2,3.4,39.0,0.2,Iris-setosa,0,"petal_length must be in range (1.0, 6.9)"
